<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Topology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
pip install networkx openpyxl

In [51]:
import networkx as netx
import pandas as pd
import numpy as np
import os
from google.colab import files

from networkx import *

def csv_to_graph(path):
  Df = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  df = Df.iloc[1:].copy()
  df['source'] = df['source'].astype(int)
  df['target'] = df['target'].astype(int)
  df['length'] = df['length'].astype(float)

  sources = df['source'].tolist()
  targets = df['target'].tolist()
  lengths = df['length'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_matrix = np.zeros((num_vertices, num_vertices))

  for source, target, length in zip(sources, targets, lengths):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_matrix[new_source, new_target] = length
    adj_matrix[new_target, new_source] = length

  return adj_matrix

def graph_topology(graph):

  topology = []

  num_nodes = graph.number_of_nodes()
  topology.append(num_nodes)

  num_edges = graph.number_of_edges()
  topology.append(num_edges)

  is_connected = netx.is_connected(graph)
  topology.append(is_connected)

  number_of_connected_components = netx.number_connected_components(graph)
  topology.append(number_of_connected_components)

  degrees = graph.degree()
  avg_degree = np.mean(graph.degree(), axis = 0)[1]
  topology.append(avg_degree)

  if netx.is_connected(graph):
    diameter = netx.diameter(graph)
  else:
    components = list(netx.connected_components(graph))
    largest_component = max(components, key=len)
    largest_component_subgraph = graph.subgraph(largest_component)
    diameter = netx.diameter(largest_component_subgraph)
  topology.append(diameter)

  edge_density = netx.density(graph)
  topology.append(edge_density)

  if netx.is_connected(graph):
    avg_shortest_path_length = netx.average_shortest_path_length(graph)
  else:
    components = list(netx.connected_components(graph))
    largest_component = max(components, key=len)
    largest_component_subgraph = graph.subgraph(largest_component)
    avg_shortest_path_length = netx.average_shortest_path_length(largest_component_subgraph)
  topology.append(avg_shortest_path_length)

  avg_clustering_coefficient = netx.average_clustering(graph)
  topology.append(avg_clustering_coefficient)

  local_clustering = list(netx.clustering(graph).values())
  std_clustering_coefficient = np.std(local_clustering)
  topology.append(std_clustering_coefficient)

  return topology

def folder_to_graphs(base_path):
  Output = []
  csv_tables = []
  for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    csv_path = os.path.join(folder_path, 'links.csv')
    csv_tables.append(folder_name)

    adj_matrix = csv_to_graph(csv_path)
    graph = netx.from_numpy_array(adj_matrix)

    Output.append(graph_topology(graph))

  labels = ['Number of Nodes', 'Number of Edges', 'Is Connected', 'Number of Connected Components', 'Average Node Degree', 'Diameter(hops)', 'Edge Density (Physical Connectivity)', 'Average Shortest Path Length (Hops)', 'Average Clustering Coefficient', 'Standard Deviation of Clustering Coefficient']
  Table = pd.DataFrame(Output, index = csv_tables ,columns = labels)
  return Table

path_to_folder = '/content/drive/MyDrive/Кола'
Table = folder_to_graphs(path_to_folder)
Table.to_excel('Topologies.xlsx', index=True)
files.download('Topologies.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>